## Environemntal

python 3.10.14  
pyqubo 

In [45]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dwave.system import DWaveSampler, EmbeddingComposite
import numpy as np
from pyqubo import Array, Constraint, Placeholder
import pandas as pd
import os
import config

## TSP solving

In [4]:
# TSP

# Number of cities
N = 10

# cities distance
w = np.array([
    [0, 29, 20, 21, 16, 31, 100, 12, 4, 31],
    [29, 0, 15, 29, 28, 40, 72, 21, 29, 41],
    [20, 15, 0, 15, 14, 25, 81, 9, 23, 27],
    [21, 29, 15, 0, 4, 12, 92, 12, 25, 13],
    [16, 28, 14, 4, 0, 16, 94, 9, 20, 16],
    [31, 40, 25, 12, 16, 0, 95, 24, 36, 3],
    [100, 72, 81, 92, 94, 95, 0, 90, 101, 99],
    [12, 21, 9, 12, 9, 24, 90, 0, 15, 25],
    [4, 29, 23, 25, 20, 36, 101, 15, 0, 35],
    [31, 41, 27, 13, 16, 3, 99, 25, 35, 0]
])


# Binary variables
x = Array.create('x', shape=(N, N), vartype='BINARY')

# cost function
cost = 0
for i in range(N):
    for j in range(N):
        for k in range(N):
            cost += w[i][j] * x[i][k] * x[j][(k+1)%N]

# Constraint

const_1 = 0
for i in range(N):
    const_1 += (np.sum(x[i]) - 1)**2

const_2 = 0
for i in range(N):
    const_2 += (np.sum(x[:, i]) - 1)**2

# const_3 = 0
# const_3 = (N - np.sum(x))**2

# Hamiltonian
H = cost + Placeholder('lambda1') * Constraint(const_1, label='const_1') + Placeholder('lambda2') * Constraint(const_2, label='const_2')

model = H.compile()

In [19]:
#parameters
feed_dict = {'lambda1': 150.0, 'lambda2': 150.0}

# QUBO
qubo, offset = model.to_qubo(feed_dict=feed_dict)

In [7]:
# Set up the D-Wave sampler
endpoint = "https://cloud.dwavesys.com/sapi/"
solver = "Advantage_system4.1"
token = config.TOKEN
anealing_time = 1000
dw_sampler = DWaveSampler(solver=solver, endpoint=endpoint, token=token, properties={"annealing_time": anealing_time})
sampler = EmbeddingComposite(dw_sampler)

In [20]:
# Solve the problem by dwave
sampleset = sampler.sample_qubo(qubo, num_reads=2000, label='TSP')

# Result

In [9]:
def cal_score(matrix):
    cost = 0
    for i in range(N):
        for j in range(N):
            for k in range(N):
                cost += w[i][j] * matrix[i][k] * matrix[j][(k+1)%N]
    return cost

In [21]:
#best solution
best_solution = sampleset.record[0]
print(best_solution.energy)

-2367.0


In [22]:
# Decode the solution
decoded_samples = model.decode_sampleset(sampleset, feed_dict=feed_dict)
for sample in decoded_samples:
    print(sample.constraints(only_broken=True))

{'const_1': (False, 1.0), 'const_2': (False, 1.0)}
{'const_1': (False, 2.0)}
{'const_2': (False, 2.0)}
{'const_1': (False, 1.0), 'const_2': (False, 1.0)}
{'const_2': (False, 1.0), 'const_1': (False, 1.0)}
{'const_2': (False, 1.0), 'const_1': (False, 1.0)}
{'const_1': (False, 2.0)}
{'const_1': (False, 2.0)}
{'const_1': (False, 1.0), 'const_2': (False, 1.0)}
{'const_2': (False, 2.0)}
{'const_2': (False, 2.0)}
{'const_1': (False, 1.0), 'const_2': (False, 1.0)}
{'const_2': (False, 2.0)}
{'const_1': (False, 1.0), 'const_2': (False, 1.0)}
{'const_1': (False, 2.0)}
{'const_1': (False, 1.0), 'const_2': (False, 1.0)}
{'const_2': (False, 1.0), 'const_1': (False, 1.0)}
{'const_2': (False, 1.0), 'const_1': (False, 1.0)}
{'const_2': (False, 2.0)}
{'const_2': (False, 1.0), 'const_1': (False, 1.0)}
{'const_2': (False, 1.0), 'const_1': (False, 1.0)}
{'const_2': (False, 1.0), 'const_1': (False, 1.0)}
{'const_2': (False, 2.0), 'const_1': (False, 2.0)}
{'const_2': (False, 1.0), 'const_1': (False, 1.0)}
{

In [54]:
print(sampleset.record)

[([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], -71735., 1, 0.04)
 ([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], -71702., 1, 0.04)
 ([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], -71628., 1, 0.02)
 ...
 ([0, 1, 0, 0, 0, 0, 1, 0, 0,

In [32]:
sampleset.to_pandas_dataframe().head()

,x[0][0],x[0][1],x[0][2],x[0][3],x[0][4],x[0][5],x[0][6],x[0][7],x[0][8],x[0][9],...,x[9][3],x[9][4],x[9][5],x[9][6],x[9][7],x[9][8],x[9][9],chain_break_fraction,energy,num_occurrences
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.02,-71655.0,1
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0.01,-71609.0,1
2,0,0,0,0,1,0,0,0,0,1,...,0,0,0,1,0,0,0,0.01,-71608.0,1
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.00,-71579.0,1
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0.01,-71548.0,1


In [23]:
best = best_solution[0].reshape(N, N).T
print(best)
print("score : ",cal_score(best))
# row : city column : time

[[0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]]
score :  354


In [18]:
second_best_solution = sampleset.record[1][0].reshape(N, N).T
print(second_best_solution)
print("score : ", cal_score(second_best_solution))

[[0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0 1 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]
score :  316
